## 딥러닝 학습

딥러닝 학습은 크게 순전파와 역전파라는 두 단계로 진행됩니다. 먼저 그림을 봅시다.

![](../pytorch_wikidocs/Static/148.jpg)

첫 번째 단계인 `순전파(forward)`는 네트워크에 훈련 데이터가 들어올 때 발생하며, 데이터를 기반으로 예측 값을 계산하기 위해 전체 신경망을 교차해 지나갑니다. 즉, 모든 뉴런이 이전 층의 뉴런에서 수신한 정보에 변환(가중합 및 활성화함수)을 적용하여 다음 층(은닉층)의 뉴런으로 전송하는 방식입니다. 네트워크를 통해 입력 데이터를 전달하며, 데이터가 모든 층을 통과하고 모든 뉴런이 계산을 완료하면 그 예측 값은 최종 층(출력층)에 도달하게 됩니다.

그 다음 손실 함수로 네트워크의 예측 값과 실제 값의 차이(손실, 오차)를 추정합니다. 이때 손실 함수 비용은 0이 이상적입니다. 따라서 `손실 함수 비용이 0에 가깝도록 하기 위해 모델이 훈련을 반복하면서 가중치를 조정`합니다. 손실(오차)이 계산되면 그 정보는 역으로 전파(출력층 -> 은닉층 -> 입력층)되기 때문에 `역전파(backpropagation)`라고 합니다. 출력층에서 시작된 손실 비용은 은닉층의 모든 뉴런으로 전파되지만, 은닉층의 뉴런은 각 뉴런이 원래 출력에 기여한 상대적 기여도에 따라(즉, 가중치에 따라) 값이 달라집니다. 좀 더 수학적으로 표현하면 예측 값과 실제 값 차이를 각 뉴런의 가중치로 미분한 후 기존 가중치 값에서 뺍니다. 이 과정을 출력층 -> 은닉층 -> 입력층 순서로 모든 뉴런에 대해 진행하여 계산된 각 뉴런 결과를 또 다시 순전파의 가중치 값으로 사용합니다.

## 딥러닝의 문제점과 해결 방안

딥러닝의 핵심은 활성화 함수가 적용된 여러 은닉층을 결합하여 비선형 영역을 표현하는 것입니다. 다음 그림과 같이 활성화 함수가 적용된 은닉층 개수가 많을수록 데이터가 분류가 잘되고 있음을 볼 수 있습니다.

![](../pytorch_wikidocs/Static/149_1.jpg)

하지만 은닉층이 많을수록 다음 `세 가지 문제점`이 생깁니다.

### 과적합 문제 발생

과적합(over-fitting)은 훈련 데이터를 과하게 학습해서 발생합니다. 일반적으로 훈련 데이터는 실제 데이터의 일부분입니다. 따라서 훈련 데이터를 과하게 학습했기 때문에 예측 값과 실제값 차이인 오차가 감소하지만, 검증 데이터에 대해서는 오차가 증가할 수 있습니다. 이러한 관점에서 과적합은 훈련 데이터에 대해 과하게 학습하여 실제 데이터에 대한 오차가 증가하는 현상을 의미합니다.

![](../pytorch_wikidocs/Static/149_2.jpg)

과적합을 해결하는 방법으로 `드롭아웃(dropout)`이 있습니다.

신경망 모델이 과적합되는 것을 피하기 위한 방법으로, 학습 과정 중 임의로 일부 노드들을 `학습에서 제외`시킵니다.

![](../pytorch_wikidocs/Static/150.jpg)

다음은 파이토치에서 드롭아웃을 구현하는 예시 코드입니다.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DropoutModel(nn.Module):
    def __init__(self):
        super(DropoutModel, self).__init__()
        self.layer1 = nn.Linear(784, 1200)
        self.dropout1 = nn.Linear(0.5) # 50%의 노드를 무작위로 선택하여 사용하지 않겠다는 의미
        self.layer2 = nn.Linear(1200, 1200)
        self.dropout2 = nn.Dropout(0.5)
        self.layer3 = nn.Linear(1200, 10)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = self.dropout1
        x = F.relu(self.layer2(x))
        x = self.dropout2(x)
        x = self.layer3(x)
        return x
        

c:\Users\aqs45\OneDrive\바탕 화면\repo\Python_AI\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 기울기 소멸 문제 발생

기울기 소멸 문제는 `은닉층이 많은 신경망`에서 주로 발생하는데, 출력층에서 은닉층으로 전달되는 `오차가 크게 줄어들어 학습이 되지 않는 현상`입니다. 즉, 기울기가 소멸되기 때문에 학습 되는 양이 0에 가까워져 학습이 더디게 진행되다 오차를 더 줄이지 못하고 그 상태로 수렴하는 현상입니다.

기울기 소멸 문제는 `시그모이드(sigmoid)`나 `하이퍼볼릭 탄젠트(tanh)` 대신 `렐루(ReLU)` 활성화 함수를 사용하면 해결할 수 있습니다.

![](../pytorch_wikidocs/Static/151_1.jpg)

### 성능이 나빠지는 문제 발생

경사 하강법은 손실 함수의 비용이 최소가 되는 지점을 찾을 때까지 기울기가 낮은 쪽으로 계속 이동시키는 과정을 반복하는데, 이때 성능이 나빠지는 문제가 발생합니다.

![](../pytorch_wikidocs/Static/151_2.jpg)

이러한 문제점을 개선하고자 확률적 경사 하강법과 미니 배치 경사 하강법을 사용합니다. 경사 하강법을 좀 더 알아보겠습니다.

![](../pytorch_wikidocs/Static/152_1.jpg)

`배치 경사 하강법(BGD)` 은 전체 데이터셋에 대한 오류를 구한 후 기울기를 한 번만 계산하여 모델의 파라미터를 업데이트 하는 방법입니다. 즉, 전체 훈련 데이터셋(total training dataset)에 대해 가중치를 편미분하는 방법입니다.

배치 경사 하강법은 한 스텝에 모든 훈련 데이터셋을 사용하므로 학습이 오래 걸리는 단점이 있습니다. 배치 경사 하강법의 학습이 오래 걸리는 단점을 개선한 방법이 확률적 경사 하강법입니다.

`확률적 경사 하강법(SGD)`은 임의로 선택한 데이터에 대해 기울기를 계산하는 방법으로 적은 데이터를 사용하므로 빠른 계산이 가능합니다. 다음 그림의 오른쪽과 같이 파라미터 변경 폭이 불안정하고, 때로는 배치 경사 하강법보다 정확도가 낮을 수 있지만 속도가 빠르다는 장점이 있습니다.

![](../pytorch_wikidocs/Static/152_2.jpg)

`미니 배치 경사 하강법(mini-batch gradient dascent)`은 전체 데이터셋을 미니 배치(mini-batch) 여러 개로 나누고, 미니 배치 한 개마다 기울기를 구한 후 그것의 평균 기울기를 이용하여 모델을 업데이트해서 학습하는 방법입니다.

![](../pytorch_wikidocs/Static/153_1.jpg)

미니 배치 경사 하강법은 전체 데이터를 계산하는 것보다 빠르며, 확률적 경사 하강법보다 안정적이라는 장점이 있기 때문에 실제로 가장 많이 사용합니다. 다음 그림의 오른쪽과 같이 파라미터 변경 폭이 확률적 경사 하강법에 비해 안정적이면서 속도도 빠릅니다.

![](../pytorch_wikidocs/Static/153_2.jpg)

파이토치에서는 다음과 같이 구현할 수 있습니다.

```py
import torch
import torch.nn as nn

from torch.utils.data import DataLoader


class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = [[1,2,3], [4,5,6], [7,8,9]]
        self.y_data = [[12], [18], [11]]
        def __len__(self):
            return len(self.x_data)
        def __getitem__(self, idx):
            x = torch.FloatTensor(self.x_data[idx])
            y = torch.FloatTensor(self.y_data[idx])
            return x, y

dataset = CustomDataset()
dataloader = DataLoader(
    dataset=dataset, # 데이터셋
    batch_size=2, # 미니 배치 크기로 2의 제곱수를 사용하겠다는 의미입니다.
    shuffle=True # 데이터를 불러올 때마다 랜덤으로 섞어서 가져옵니다.
```

## 옵티마이저

확률적 경사 하강법의 파라미터 변경 폭이 불안정한 문제를 해결하기 위해 `학습 속도` 와 `운동량`을 조정하는 `옵티마이저(optimizer)`를 적용해 볼 수 있습니다.

![](../pytorch_wikidocs/Static/154.jpg)



### 아다그라드(Adagrad, Adaptive gradient)
아다그라드는 변수(가중치)의 업데이트 횟수에 따라 학습률을 조정하는 방법입니다. 아다그라드는 많이 변화하지 않는 변수들의 학습률은 크게 하고, 많이 변화하는 변수들의 학습률은 작게 합니다. 즉, 많이 변화한 변수는 최적 값에 근접했을 것이라는 가정하에 작은 크기로 이동하면서 세밀하게 값을 조정하고, 반대로 적게 변화한 변수들은 학습률을 크게 하여 빠르게 오차 값을 줄이고자 하는 방법입니다.

예를 들어 파이토치에서는 아다그리드를 다음과 같이 구현할 수 있습니다.

```py
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01)
```

하지만 아다그라드는 기울기가 0에 수렴하는 문제가 있어 사용하지 않으며, 대신에 `알엠에스프롭`을 사용합니다.

### 아다델다(Adadelta, Adaptive delta)
아다델타는 아다그라드 G 값이 커짐에 따라 학습이 멈추는 문제를 해결하기 위해 등장한 방법입니다. 아다델타는 아다그릳의 수식에서 학습률(n)을 D함수로 변환했기때문에 학습률에 대한 하이퍼파라미터가 필요하지 않습니다.

예를 들어 파이토치에서는 아다델타를 다음과 같이 구현할 수 있습니다.

```py
optimizer = torch.optim.Adadelta(model.parameters(), lr=1.0)
```

### 알엠에스프롭(RMSProp)
알엠에스프롭은 아다그라드의 G(i)값이 무한히 커지는 것을 방지하고자 제안된 방법입니다.

예를 들어 파이토치에서는 알엠에스프롭을 다음과 같이 구현할 수 있습니다.

```py
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.01)
```

## 모멘텀(Momentum)
경사 하강법과 마찬가지로 매번 기울기를 구하지만, 가중치를 수정하기 전에 이전 수정 방향(+, -)을 참고하여 같은 방향으로 일정한 비율만 수정하는 방법입니다. 수정이 양(+)의 방향과 음(-)의 방향으로 순차적으로 일어나는 지그재그 현상이 줄어들고, 이전 이동 값을 고려하여 일정 비율만큼 다음 값을 결정하므로 관성 효과를 얻을 수 있는 장점이 있습니다. 모멘텀은 SGD(확률적 경사 하강법)와 함께 사용합니다.

### SGD 모멘텀(SGD with Momentum)
SGD모멘텀은 확률적 경사 하강법에서 기울기를 속도로 대체하여 사용하는 방식으로, 이전 속도의 일정 부분을 반영합니다. 즉, 이전에 학습했던 속도와 현재 기울기를 반영해서 가중치를 구합니다.

예를 들어 파이토치에서는 다음과 같이 모멘텀을 구현할 수 있습니다.

```py
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
```

`momentum` 값은 0.9에서 시작하며 0.95, 0.99처럼 조금씩 증가하면서 사용합니다.


### 아담(Adam, Adaptive Moment Estimation)
아담은 모멘텀과 알엠에스프롭의 장점을 결합한 경사 하강법입니다. 알엠에스프롭의 특징인 기울기의 제곱을 지수 평균한 값과 모멘텀 특징인 v(i)를 수식에 활용합니다. 즉, 알엠에스프롭의 G 함수와 모멘텀의 v(i)를 사용하여 가중치를 업데이트합니다.

예를 들어 파이토치에서는 다음과 같이 아담을 구현할 수 있습니다.

```py
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
```


## 딥러닝을 사용할 때 이점

그럼 딥러닝을 사용할 때 이점에서 어떤 것이 있을까요?

#### 특성 추출

컴퓨터가 입력받은 데이터를 분석하여 일정한 패턴이나 규칙을 찾아내려면 사람이 인지하는 데이터를 컴퓨터가 인지할 수 있는 데이터로 변환해 주어야 합니다. 이때 데이터별로 어떤 특징을 가지고 있는지 찾아내고, 그것을 토대로 데이터를 벡터로 변환하는 작업을 `특성 추출(feature extraction)` 이라고 합니다.

딥러닝이 활성화되기 이전에 많이 사용되었던 머신 러닝 알고리즘은 SVM, 나이브 베이즈(Naive Bayes), 로지스틱 회귀의 특성 추출은 매우 복잡하며 수집된 데이터에 대한 전문 지식(예를 들어 제조, 의료 등 수집된 데이터의 도메인 분야에 대한 지식)이 필요했습니다. 하지만 딥러닝에서는 이러한 특성 추출 과정을 알고리즘에 통합시켰습니다. 데이터 특성을 잘 잡아내고자 은닉층을 깊게 쌓는 방식으로 파라미터를 늘린 모델 구조 덕분입니다.

#### 빅데이터의 효율적 활용

딥러닝을 사용할 때의 이점으로 특성 추출이 있다고 했습니다. 즉, 딥러닝에서는 특성 추출을 알고리즘에 통합시켰다고 했는데, 이것이 가능한 이유는 빅데이터 때문입니다. 딥러닝 학습을 이용한 특성 추출은 데이터 사례가 많을수록 성능이 향상되기 때문입니다.

다른 말로 표현하면 확보된 데이터가 적다면 딥러닝의 성능 향상을 기대하기 힘들기 때문에 머신 러닝을 고려해 보아야 합니다.

